In [38]:
#lib para facilitar o uso de dataframes
import pandas as pd
#lib para facilitar alguns cálculos
import numpy as np
#Lib para auxiliar o processo de count
from collections import Counter

import itertools

In [39]:
#lendo o csv com o dataset pronto
df_voting = pd.read_csv('data/cleaned_voting.csv')

In [40]:
#Confirmando o formato dos dados
df_voting.head()

,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15,p_16,partido_democrat,partido_republican
0,n,n,n,y,y,y,n,n,n,y,n,y,y,y,n,n,n,y
1,n,n,y,n,n,y,y,y,y,n,y,n,n,n,y,y,y,n
2,y,y,n,n,n,y,y,n,n,n,y,y,n,y,n,y,y,n
3,y,y,y,n,y,y,n,n,n,n,y,n,y,y,n,n,y,n
4,n,n,n,y,y,n,n,n,n,n,n,y,n,y,n,n,n,y


# Regras de associacao

Regras de associacao sao tecnicas de mineiracao de dados usadas para descobrir associacoes interessantes entre atributos de um determinado banco de dados.
A definicao classica de regras de associacao foi apresentada em Agrawal, Imielinski, & Swami (1993) and Han & Kamber, (2006)), e e definida como o seguinte:

Seja T {t1,t2...tn} um conjunto de transacoes e I {i1,i2...in} um conjunto de itens, definimos D como o dado relevante para a tarefa sendo um conjunto de transacoes onde cada T e um conjunto de itens de maneira que T seja um subconjunto de I.

Uma regra de associacao tem a forma X -> Y, onde X e Y sao conjuntos disjuntos

Em nosso dataset as transacoes T representao o conjunto de votos de cada representante, e os Itens representao a reposta ao voto para cada projeto.

O algoritmo aprior é usado para encontrar conjuntos frequentes de items em databases como o nosso.
Ele funciona com a afirmativa que:
> Um sub conjunto de um conjunto frequente provavelmente e trambem um conjunto frequente

Por exemplo:
   se {i1,i2} é um conjunto frequente entao provavelmente {i1} e {i2} tambem devem ser conjuntos de items frequentes
   
- O algoritmo faz varias iteracoes em busca de itens frequentes e usa esses itens frequentes para gerar as regras de associacao

Abaixo vemos a implementacao do algoritmo passo a passo

---

Antes de comecarmos com o algoritmo precisamos realizar a definicao das metricas

In [41]:
'''
Variavel global para definicao de limite superior para suporte
'''
SUP_THRESHOLD = 0.50

In [42]:
def get_suporte(qtd_X, qtd_total):
    '''
    suporte = Numero de transacoes que um item aparece
              ---------------------------------------
              Numero total de transacoes
    '''
    return (qtd_X*1.0)/qtd_total

def get_confianca():
    pass

In [43]:
def filter_candidates(candidates, df):
    if len(candidates) < 1:
        return df
    
    # seleciona elementos canditados presentes nas transacoes
    query = ' & '.join(['{} == "y"'.format(k) for k in candidates])
    filtered_df = df.query(query)
    
    return filtered_df

In [44]:
# teste
#filter_candidates([],df_voting)

,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15,p_16,partido_democrat,partido_republican
0,n,n,n,y,y,y,n,n,n,y,n,y,y,y,n,n,n,y
1,n,n,y,n,n,y,y,y,y,n,y,n,n,n,y,y,y,n
2,y,y,n,n,n,y,y,n,n,n,y,y,n,y,n,y,y,n
3,y,y,y,n,y,y,n,n,n,n,y,n,y,y,n,n,y,n
4,n,n,n,y,y,n,n,n,n,n,n,y,n,y,n,n,n,y
5,y,n,n,y,y,y,n,n,n,y,n,y,y,y,n,n,n,y
6,y,n,y,n,n,n,y,y,y,n,n,n,n,n,n,y,y,n
7,n,y,n,y,y,y,n,n,n,y,n,y,y,y,n,y,n,y
8,n,n,n,y,y,y,n,n,n,y,n,y,y,y,n,y,n,y
9,y,n,y,n,n,n,y,y,y,y,n,n,n,n,y,y,y,n


## Passo 1

No primeiro passo vamos crair uma tabela com todos os counts individuais de cada votacao
positiva para cada item

In [114]:
def itens_distintos(df):
    '''
    1. Conta a quantidade de y e n em todas as colunas, pegando apenas y no final
    e Transpondo para melhorar o entendimento da tabela
    2. Adiciona uma coluna com o suporte calculado
    3. Passa o threshold de suporte retornando apenas os itens acima do suporte
    '''
    #1.
    countings = pd.DataFrame(df.apply(pd.Series.value_counts).T['y'])
    #2.
    countings['suporte'] = countings.apply(lambda k: get_suporte(k['y'], df_voting.shape[0]),axis=1)
    #3.
    countings = countings.loc[countings['suporte'] >= SUP_THRESHOLD]
    
    return list(countings.T)

def apriori():
    res = []
    


In [58]:
#candidatos = itens_distintos(df_voting)

In [105]:
def apriori():
    res = []
    # calcula o primeiro L1 {large 1-itemsets}
    candidates = itens_distintos(df_voting)
    res.append(candidates)
    
    # para cada possibilidade de k-itemsets
    for k in range(2, df_voting.shape[1]):
        new_candidates = candidate_gen(candidates, k)
        df_tmp = filter_candidates(new_candidates, df_voting)
        candidates = itens_distintos(df_tmp)
        
        if len(candidates) < 1 :
            break
        res.append(candidates)
        
    return res

In [92]:
def gera_combinacao(candidatos, k):
    '''
    A partir de uma lista gera todas as combinacoes possiveis de tamanho k
    Ex:
    para k = 2
    F3 = {{1, 2, 3}, {1, 2, 4}, {1, 3, 4}, {1, 3, 5}, {2, 3, 4}}.
    Alguns grupos gerados
    1,2,3 e 1,2,4
    1,2,3 e 1,3,4
    1,2,3 e 1,3,5
    ...
    '''
    #geramos todas as combinacoes possiveis de i para o resto da lista
    #usando a funcao combinations de itertools
    return [list(x) for x in itertools.combinations(candidatos, k)]

In [111]:
def candidate_gen(candidates, k):
    Ck = []
    
    combinations = gera_combinacao(candidates, k)
    print combinations
    for f1 in combinations:
        for f2 in combinations:
            if f1[-1] < f2[-1]:
                c = set(f1 + f2)
                c_subsets = gera_combinacao(list(c), k)
                is_subset = True
                for i in c_subsets:
                    if i not in candidates:
                        is_subset = False
                        break
                if is_subset:
                    Ck.append(c)
            
    
    return Ck

In [113]:
candidate_gen([[1,2,3],[1,2,4],[2,3,4],[1,3,4],[1,3,5]],4)

[[[1, 2, 3], [1, 2, 4], [2, 3, 4], [1, 3, 4]], [[1, 2, 3], [1, 2, 4], [2, 3, 4], [1, 3, 5]], [[1, 2, 3], [1, 2, 4], [1, 3, 4], [1, 3, 5]], [[1, 2, 3], [2, 3, 4], [1, 3, 4], [1, 3, 5]], [[1, 2, 4], [2, 3, 4], [1, 3, 4], [1, 3, 5]]]


TypeError: unhashable type: 'list'

In [106]:
apriori()

['p_2', 'p_3', 'p_6', 'p_7', 'p_8', 'p_9', 'p_10', 'p_13', 'p_14', 'p_16', 'partido_democrat']
